# 국민재난안전포털 웹 크롤러 만들기

In [7]:
from requests import Session
from requests.compat import urljoin
from bs4 import BeautifulSoup

In [59]:
session = Session()

base_url = 'http://www.safekorea.go.kr/'
main_url = 'idsiSFK/neo/ext/json/main/menuList.json'

main_resp = session.get(urljoin(base_url, main_url))
result = main_resp.json()

disaster_service_url = ''
disaster_menu_seq = 1

# 재난문자 url과 params인 munuSeq 구하기
for cnt in range(0,len(result)):
    if cnt > 0 and result[cnt]['MENU_NM'] == '재난문자':
        disaster_service_url = 'idsiSFK' + result[cnt]['SERVICE_URL']
        disaster_menu_seq = result[cnt]['MENU_SEQ']
        break
        
#disaster_service_url, disaster_menu_seq

## 아래 코드 사용 불가
- post 방식의 재난문자 json request시 500 error 발생

In [75]:
session.cookies.clear()
disaster_resp = session.request('GET', urljoin(base_url, disaster_service_url), params={'menuSeq':disaster_menu_seq})
#dom = BeautifulSoup(disaster_resp.text, 'html.parser')
referer = disaster_resp.request.url

#새 쿠키값 
cookiedict = {}
for _ in session.cookies.items():
    cookiedict[_[0]] = _[1]

headers = {
    'Referer' : referer,
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}

#쿠키 날짜 value는 당일 날짜로 변경해야함, 실시간 크롤링시
cookies = {
    'JSESSIONID':cookiedict['JSESSIONID'],
    'elevisor_for_j2ee_uid':cookiedict['elevisor_for_j2ee_uid'],
    'e4d3f82b588e6cf132ee4765403cd800':cookiedict['e4d3f82b588e6cf132ee4765403cd800'],
    '3053195ebb01870828aed202f55dc974':'76ff93199833d41c33e1b89a9862350b',
    'popup_1':'20200802',
    'popup_2':'20200802',
    '_ga':'GA1.3.522710010.1595930482',
    '_gid':'GA1.3.1758721736.1596362546'
}

#session.request지만 누락된 쿠키값이 있어서 따로 다시 입력
disaster_list = 'idsiSFK/bbs/user/selectBbsList.do'
disaster_list_resp = session.request('POST', urljoin(base_url, disaster_list), headers=headers, cookies=cookies)

In [76]:
disaster_list_resp.request, disaster_list_resp.reason, disaster_list_resp.status_code, disaster_list_resp.headers

(<PreparedRequest [POST]>,
 'Internal Server Error',
 500,
 {'Content-Encoding': 'gzip', 'Content-Type': 'text/html;charset=ISO-8859-1', 'Content-Length': '254', 'Date': 'Sun, 02 Aug 2020 14:56:23 GMT'})

# 셀레니움 활용 실시간 문자 수집
- 셀레니움 사용 이유: 재난문자 페이지의 json request 불가(inter server error, 500 error)

In [77]:
from selenium import webdriver

In [83]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)

uu = urljoin(base_url, disaster_service_url)
uuu = uu + '&menuSeq=' + str(disaster_menu_seq)
driver.get(uuu)

#### (수정1) 재난문자 첫 페이지 => 새로운 재난문자 생길 경우 => 새로운 문자만 접근
- list에 첫 페이지의 모든 재난문자 제목 insert
- 맨 밑의 문자부터 가장 최근 문자 순으로 체크
- 새로운 문자가 있을 경우, 가장 나중에 갱신된 문자~처음 갱신된 문자까지 접근

#### (처음) 재난문자 첫 페이지 => 10개 재난문자 가져오기
- 재난문자 하나씩 접근하므로 중간에 재난문자 갱신이 되면 똑같은 문자 접근할 수 있음

In [156]:
import time

run_count = 0
last_chk_topic = []
new_chk_topic = []
def run_chk_msg():
    global run_count
    global chk_topic
    
    print('run_count: {0}'.format(run_count))
    print('chk_msg!')
    
    for idx in range(0, 10): # last_재난문자
        chk_msg = '//tbody[@id="bbs_tr"]/tr[@id="bbs_tr_{0}_apiData1"]/td[2]'.format(idx)
        last_chk_topic.insert(idx, driver.find_element_by_xpath(chk_msg).text)
        print(last_chk_topic[idx])
        
    # 현재 켜져 있는 웹페이지에서 새로고침 후 새로운 재난문자 접근
    # 나중에 기준 정하기. 웹 서비스로 처음 시각화되는 재난문자를 어떤 것으로 둘지(지금은 시각화되고 있음을 가정)
    driver.refresh()
    new_msg_idx = -1 # 바뀐 문자 index
    for idx in range(9, -1, -1): # new_재난문자 체크
        cur_msg_path = '//tbody[@id="bbs_tr"]/tr[@id="bbs_tr_{0}_apiData1"]/td[2]'.format(idx)
        cur_msg = driver.find_element_by_xpath(cur_msg_path)
        new_exist = True
        for chk in range(9, -1, -1):
            if last_chk_topic[chk] == cur_msg.text: # 같은 재난문자이면 break
                new_exist = False
                break
        if new_exist == True:
            # 재난문자가 새로 발신된 경우, for문 break
            new_msg_idx = idx
            print('new 재난문자: {0}개'.format(new_msg_idx+1))
            break
    
    print('new_msg_idx: {0}'.format(new_msg_idx))
    if new_msg_idx >= 0: # 새로운 문자가 발신된 경우
        for cnt in range(0, new_msg_idx+1):
            # cnt 번째 페이지, 첫 번째 재난 문자 path
            disaster_path = '//tbody[@id="bbs_tr"]/tr[@id="bbs_tr_{0}_apiData1"]/td[2]'.format(cnt)
            result_topic = driver.find_element_by_xpath(disaster_path)
            print((cnt+1), '번째 재난문자:', result_topic.text)

            # cnt 번째 재난문자 링크 이동
            first_a = disaster_path + '/a'
            last_url = driver.current_url
            driver.find_element_by_xpath(first_a).click()
            driver.implicitly_wait(3)

            # 재난문자 상세내용 확인
            disaster_context = '//div[@id="content"]/div[@id="content"]/div/div[@class="boardView"]/div[@class="boardView_context"]'
            result_context = driver.find_element_by_xpath(disaster_context)
            print(result_context.text)
            print('\n')
            driver.implicitly_wait(3)

            # 이전 페이지로 돌아가기
            driver.get(last_url)
            
    run_count += 1

In [157]:
while True:
    if run_count > 0:
        time.sleep(30)
    run_chk_msg()
#     if run_count > 50:
#         print('end')
#         break

run_count: 0
chk_msg!
2020/08/04 15:44:18 재난문자[화천군청]
2020/08/04 15:27:48 재난문자[세종시청]
2020/08/04 15:14:52 재난문자[철원군청]
2020/08/04 15:13:24 재난문자[양구군청]
2020/08/04 15:12:54 재난문자[양구군청]
2020/08/04 15:07:15 재난문자[인제군청]
2020/08/04 14:50:12 재난문자[고창군청]
2020/08/04 14:45:05 재난문자[천안시청]
2020/08/04 14:22:15 재난문자[제천시청]
2020/08/04 14:21:48 재난문자[화천군청]
new 재난문자: 2개
new_msg_idx: 1
1 번째 재난문자: 2020/08/04 16:11:14 재난문자[철원군청]
[철원군청]금일 근남면 육단리 육단2교의 범람이 우려되오니 도로 통행제한 중에 있습니다. 도로 침수되여 통행이 어려우니 우회바랍니다.

-송출지역-
강원도 철원군


2 번째 재난문자: 2020/08/04 15:56:00 재난문자[화천군청]
[화천군청] 금일 상서면 다목리 수피령고개 일원 집중호우로 통행제한 중에 있습니다. 도로 침수 및 토사유출로 통행이 어려우니 우회바랍니다.

-송출지역-
강원도 화천군


run_count: 1
chk_msg!
2020/08/04 16:11:14 재난문자[철원군청]
2020/08/04 15:56:00 재난문자[화천군청]
2020/08/04 15:44:18 재난문자[화천군청]
2020/08/04 15:27:48 재난문자[세종시청]
2020/08/04 15:14:52 재난문자[철원군청]
2020/08/04 15:13:24 재난문자[양구군청]
2020/08/04 15:12:54 재난문자[양구군청]
2020/08/04 15:07:15 재난문자[인제군청]
2020/08/04 14:50:12 재난문자[고창군청]
2020/08/04 14:45:05 재난문자[천안시청]
new_msg_idx: -1
run_count: 2
chk_ms

KeyboardInterrupt: 